In [18]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
from nltk import pos_tag, word_tokenize
import re

Populating the interactive namespace from numpy and matplotlib


In [74]:
df = pd.read_csv('data.csv', index_col=0)
df.head()

,date,message,name
0,"April 12, 2007 2:52AM",Hi Pat:You forgot the Chrysler Sebring,motownusa
1,"April 12, 2007 3:33AM",I'm sure some folks would appreciate having th...,exshoman
2,"April 12, 2007 2:51PM",You can try to revive this topic but without b...,targettuning
3,"April 12, 2007 4:43PM",Model vs. model is exactly what we're here for...,pat
4,"April 13, 2007 7:49PM",The Altima is my favorite of the bunch. It is...,perna


In [196]:
model_df = pd.read_csv('Cars make model list.csv')
model_df['Replace'] = model_df['Replace'].str.lower()
model_df['Search'] = model_df['Search'].str.lower()
model_dict = model_df.set_index('Search')['Replace'].to_dict()
model_df.head()

,Replace,Search
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx


In [197]:
from nltk.corpus import stopwords
StopWordslist = stopwords.words("english")

In [198]:

def string_process(s):
    s=str(s)
    lower = s.lower()
    for key, value in model_dict.items():
        lower= lower.replace(key, value)
    words = lower.split()
    refined = []
    for i in words:
        if i not in StopWordslist:
            refined.append(re.sub("[^\w]+", "", i))
    sentence = ' '.join([word for word in refined])
    return(sentence)

In [199]:
df['text']=df['message'].map(string_process)

/Users/Jack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [200]:
model_set=set(model_dict.values())-{'car','seat','sedan','problem'}
count_df=DataFrame(df['text'])
for i in model_set:
    count_df[i]=0

In [201]:
def count_brand():
    for num in range(len(count_df['text'])):
        for i in model_set:
            if i in count_df['text'][num]:
                count_df.loc[num,i]=1
        if num%603==0:
            print 'Task ',num/60.3,'% Finished'
    print 'Total Finished'
count_brand()

Task  0.0 % Finished
Task  10.0 % Finished
Task  20.0 % Finished
Task  30.0 % Finished
Task  40.0 % Finished
Task  50.0 % Finished
Task  60.0 % Finished
Task  70.0 % Finished
Task  80.0 % Finished
Task  90.0 % Finished
Task  100.0 % Finished
Total Finished


In [192]:
count_df.drop('text',1, inplace=True)
count_df.loc['Row_sum'] = count_df.apply(lambda x: x.sum())
count_series=count_df.T['Row_sum']

In [193]:
count_series.sort_values(ascending=False)[:10]

honda        2600
ford         1624
toyota       1189
hyundai       788
mazda         745
nissan        721
chevrolet     375
chrysler      334
saturn        334
dodge         290
Name: Row_sum, dtype: int64

# Task A

In [194]:
count_df

,toyata,subaru,buick,audi,mercury,mitsubishi,chevrolet,suzuki,volkswagen,dodge,...,lincoln,acura,nissan,toyota,nissan.,volvo,chrysler,hyundai.,"hyundai,",volkwagen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
